In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
loan_df = pd.read_csv("lending_data.csv")

# Review the DataFrame
loan_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = loan_df["loan_status"]

# Separate the X variable, the features
X = loan_df.drop(columns="loan_status")

In [4]:
# Review the y variable Series
y[0:5]

,loan_status
0,0
1,0
2,0
3,0
4,0


In [5]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

,count
loan_status,
0,75036
1,2500


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model.fit(x_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
y_pred = lr_model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_score = balanced_accuracy_score(y_test, y_pred)
print(balanced_score)

0.967989851522121


In [11]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, y_pred)

array([[18655,   110],
       [   36,   583]])

In [12]:
# shows 18,655 healthy loans predicted and 583 unhealthy loans predicted. False positives predicted = 110 and false negatives predicted = 36

In [13]:
# Print the classification report for the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.94      0.89       619

    accuracy                           0.99     19384
   macro avg       0.92      0.97      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** For healthy loans the high value of precision, recall, and F1 score indicate that the model does an excellenct job at predicting values. For unhealthy loans (at-risk), the precision, recall, and F1 values are lower, but still relatively high. This indicates that the model also does a good job at predicting unhealthy values (84% accuracy), but just not as good at when predicting healty loan values (100% accuracy).

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)

In [15]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

,count
loan_status,
0,56271
1,56271


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
ros_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
ros_model.fit(x_resampled, y_resampled)

# Make a prediction using the testing data
ros_predictions = ros_model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [17]:
# Print the balanced_accuracy score of the model
ros_balanced_score = balanced_accuracy_score(y_test, ros_predictions)
print(ros_balanced_score)

0.9935981855334257


In [18]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, ros_predictions)

array([[18646,   119],
       [    4,   615]])

In [19]:
# Print the classification report for the model
print(classification_report(y_test, ros_predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Using the oversampled data results in an improved recall value to 0.99 from initially 0.94. This indicates that the probability of predicted a bad loan is increased from 94% to 99%. For the oversampled data, there were only 4 false positives and 119 false negatives predicted out of the 19,384 total loans. This model therefore would be extremly useful in predictions loans that could default, and would be therefule useful and reliable for the bank to avoid such high-risk loans.